In [2]:
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
df = pd.read_csv('dataset.csv')
df.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0.387851,0.102582,1912.506881,2361.474653,4776.862782,0.058426,-182.638275,90.676788,14.119084,67.549828,...,5.811708,-5.149533,0.192984,-14.480762,6.862884,-4.442362,-2.986888,-4.816248,5.400325,blues
1,0.412095,0.282645,3194.109244,3052.840219,6993.491045,0.159380,24.490849,68.962845,-4.211277,-1.029357,...,-0.008605,2.069138,1.552778,4.000855,2.817995,2.201693,1.577579,-0.471415,0.527109,pop
2,0.409125,0.179450,3262.545498,3046.902288,6828.042045,0.135866,-112.386246,59.745834,14.492916,19.199791,...,-0.740702,1.880871,-0.701747,-2.300871,-3.492096,-3.894060,-3.386057,-2.424497,-2.087777,pop
3,0.372834,0.208743,1345.850763,1841.748033,2939.265560,0.041099,-127.946869,134.443878,1.027332,54.440975,...,11.043714,-8.385376,2.111019,-6.338261,0.997383,-5.325869,-0.842191,-4.665996,-4.787677,rock
4,0.262947,0.062617,1313.698893,1371.049973,2397.052172,0.081098,-228.917129,153.983124,-30.688818,20.806396,...,-0.724534,-3.109805,-3.437935,-2.629030,0.719196,-1.973606,-2.513512,1.571475,-1.484867,classical


In [4]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

In [5]:
dataset = df.values
X = dataset[:,0:26].astype(np.float32)
Y_obj = dataset[:,26]

In [6]:
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)
Y = Y.astype(np.float32)
Y = tf.keras.utils.to_categorical(Y)

In [7]:
df.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0.387851,0.102582,1912.506881,2361.474653,4776.862782,0.058426,-182.638275,90.676788,14.119084,67.549828,...,5.811708,-5.149533,0.192984,-14.480762,6.862884,-4.442362,-2.986888,-4.816248,5.400325,blues
1,0.412095,0.282645,3194.109244,3052.840219,6993.491045,0.159380,24.490849,68.962845,-4.211277,-1.029357,...,-0.008605,2.069138,1.552778,4.000855,2.817995,2.201693,1.577579,-0.471415,0.527109,pop
2,0.409125,0.179450,3262.545498,3046.902288,6828.042045,0.135866,-112.386246,59.745834,14.492916,19.199791,...,-0.740702,1.880871,-0.701747,-2.300871,-3.492096,-3.894060,-3.386057,-2.424497,-2.087777,pop
3,0.372834,0.208743,1345.850763,1841.748033,2939.265560,0.041099,-127.946869,134.443878,1.027332,54.440975,...,11.043714,-8.385376,2.111019,-6.338261,0.997383,-5.325869,-0.842191,-4.665996,-4.787677,rock
4,0.262947,0.062617,1313.698893,1371.049973,2397.052172,0.081098,-228.917129,153.983124,-30.688818,20.806396,...,-0.724534,-3.109805,-3.437935,-2.629030,0.719196,-1.973606,-2.513512,1.571475,-1.484867,classical


In [6]:
# 변형 객체 생성
minmax_scaler = MinMaxScaler()
# 훈련데이터의 모수 분포 저장
minmax_scaler.fit(X)
# 훈련 데이터 스케일링
X = minmax_scaler.transform(X)
# 테스트 데이터의 스케일링

In [7]:
n_fold = 10

mskf = MultilabelStratifiedKFold(n_splits=n_fold, 
                    shuffle=True, random_state=seed)

In [8]:
accuracy=[]

In [9]:
for train, test in mskf.split(X, Y):
    model = Sequential()
    model.add(Dense(128, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs = 500)
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)

Train on 900 samples
Epoch 1/500
900/900 [==============================] - 1s 1ms/sample - loss: 2.1897 - accuracy: 0.2378
Epoch 2/500
900/900 [==============================] - 0s 84us/sample - loss: 1.8964 - accuracy: 0.3556
Epoch 3/500
900/900 [==============================] - 0s 79us/sample - loss: 1.7129 - accuracy: 0.3589
Epoch 4/500
900/900 [==============================] - 0s 80us/sample - loss: 1.6034 - accuracy: 0.3944
Epoch 5/500
900/900 [==============================] - 0s 78us/sample - loss: 1.5416 - accuracy: 0.4311
Epoch 6/500
900/900 [==============================] - 0s 78us/sample - loss: 1.4812 - accuracy: 0.4167
Epoch 7/500
900/900 [==============================] - 0s 85us/sample - loss: 1.4195 - accuracy: 0.4789
Epoch 8/500
900/900 [==============================] - 0s 80us/sample - loss: 1.3822 - accuracy: 0.4956
Epoch 9/500
900/900 [==============================] - 0s 79us/sample - loss: 1.3495 - accuracy: 0.5267
Epoch 10/500
900/900 [======================

900/900 [==============================] - 0s 74us/sample - loss: 0.4574 - accuracy: 0.8478
Epoch 156/500
900/900 [==============================] - 0s 75us/sample - loss: 0.4392 - accuracy: 0.8533
Epoch 157/500
900/900 [==============================] - 0s 76us/sample - loss: 0.4659 - accuracy: 0.8344
Epoch 158/500
900/900 [==============================] - 0s 75us/sample - loss: 0.4417 - accuracy: 0.8556
Epoch 159/500
900/900 [==============================] - 0s 73us/sample - loss: 0.4449 - accuracy: 0.8522
Epoch 160/500
900/900 [==============================] - 0s 74us/sample - loss: 0.4765 - accuracy: 0.8389
Epoch 161/500
900/900 [==============================] - 0s 74us/sample - loss: 0.4417 - accuracy: 0.8533
Epoch 162/500
900/900 [==============================] - 0s 74us/sample - loss: 0.4322 - accuracy: 0.8567
Epoch 163/500
900/900 [==============================] - 0s 76us/sample - loss: 0.4307 - accuracy: 0.8522
Epoch 164/500
900/900 [==============================] - 0s 

900/900 [==============================] - 0s 76us/sample - loss: 0.1286 - accuracy: 0.9722
Epoch 310/500
900/900 [==============================] - 0s 80us/sample - loss: 0.1209 - accuracy: 0.9711
Epoch 311/500
900/900 [==============================] - 0s 78us/sample - loss: 0.1273 - accuracy: 0.9756
Epoch 312/500
900/900 [==============================] - 0s 78us/sample - loss: 0.1247 - accuracy: 0.9767
Epoch 313/500
900/900 [==============================] - 0s 75us/sample - loss: 0.1219 - accuracy: 0.9767
Epoch 314/500
900/900 [==============================] - 0s 75us/sample - loss: 0.1235 - accuracy: 0.9722
Epoch 315/500
900/900 [==============================] - 0s 78us/sample - loss: 0.1146 - accuracy: 0.9722
Epoch 316/500
900/900 [==============================] - 0s 78us/sample - loss: 0.1688 - accuracy: 0.9544
Epoch 317/500
900/900 [==============================] - 0s 77us/sample - loss: 0.1377 - accuracy: 0.9589
Epoch 318/500
900/900 [==============================] - 0s 

900/900 [==============================] - 0s 72us/sample - loss: 0.0234 - accuracy: 0.9989
Epoch 464/500
900/900 [==============================] - 0s 73us/sample - loss: 0.0321 - accuracy: 0.9978
Epoch 465/500
900/900 [==============================] - 0s 71us/sample - loss: 0.0387 - accuracy: 0.9889
Epoch 466/500
900/900 [==============================] - 0s 72us/sample - loss: 0.0257 - accuracy: 0.9978
Epoch 467/500
900/900 [==============================] - 0s 73us/sample - loss: 0.0320 - accuracy: 0.9978
Epoch 468/500
900/900 [==============================] - 0s 77us/sample - loss: 0.0411 - accuracy: 0.9933
Epoch 469/500
900/900 [==============================] - 0s 81us/sample - loss: 0.0348 - accuracy: 0.9967
Epoch 470/500
900/900 [==============================] - 0s 83us/sample - loss: 0.0716 - accuracy: 0.9811
Epoch 471/500
900/900 [==============================] - 0s 82us/sample - loss: 0.0604 - accuracy: 0.9867
Epoch 472/500
900/900 [==============================] - 0s 

900/900 [==============================] - 0s 75us/sample - loss: 0.7279 - accuracy: 0.7622
Epoch 90/500
900/900 [==============================] - 0s 80us/sample - loss: 0.7279 - accuracy: 0.7544
Epoch 91/500
900/900 [==============================] - 0s 79us/sample - loss: 0.7092 - accuracy: 0.7589
Epoch 92/500
900/900 [==============================] - 0s 77us/sample - loss: 0.7380 - accuracy: 0.7500
Epoch 93/500
900/900 [==============================] - 0s 82us/sample - loss: 0.7084 - accuracy: 0.7556
Epoch 94/500
900/900 [==============================] - 0s 79us/sample - loss: 0.6997 - accuracy: 0.7622
Epoch 95/500
900/900 [==============================] - 0s 79us/sample - loss: 0.7002 - accuracy: 0.7667
Epoch 96/500
900/900 [==============================] - 0s 81us/sample - loss: 0.7108 - accuracy: 0.7544
Epoch 97/500
900/900 [==============================] - 0s 79us/sample - loss: 0.6852 - accuracy: 0.7711
Epoch 98/500
900/900 [==============================] - 0s 82us/samp

...

900/900 [==============================] - 0s 93us/sample - loss: 0.2380 - accuracy: 0.9256
Epoch 266/500
900/900 [==============================] - 0s 90us/sample - loss: 0.2132 - accuracy: 0.9311
Epoch 267/500
900/900 [==============================] - 0s 93us/sample - loss: 0.2448 - accuracy: 0.9156
Epoch 268/500
900/900 [==============================] - 0s 91us/sample - loss: 0.2603 - accuracy: 0.9056
Epoch 269/500
900/900 [==============================] - 0s 86us/sample - loss: 0.2083 - accuracy: 0.9433
Epoch 270/500
900/900 [==============================] - 0s 91us/sample - loss: 0.1967 - accuracy: 0.9589
Epoch 271/500
900/900 [==============================] - 0s 95us/sample - loss: 0.2095 - accuracy: 0.9433
Epoch 272/500
900/900 [==============================] - 0s 86us/sample - loss: 0.2081 - accuracy: 0.9433
Epoch 273/500
900/900 [==============================] - 0s 95us/sample - loss: 0.2100 - accuracy: 0.9389
Epoch 274/500
900/900 [==============================] - 0s 

900/900 [==============================] - 0s 85us/sample - loss: 0.0413 - accuracy: 0.9967
Epoch 420/500
900/900 [==============================] - 0s 90us/sample - loss: 0.0367 - accuracy: 0.9978
Epoch 421/500
900/900 [==============================] - 0s 78us/sample - loss: 0.0383 - accuracy: 0.9967
Epoch 422/500
900/900 [==============================] - 0s 80us/sample - loss: 0.0402 - accuracy: 0.9933
Epoch 423/500
900/900 [==============================] - 0s 84us/sample - loss: 0.0722 - accuracy: 0.9844
Epoch 424/500
900/900 [==============================] - 0s 82us/sample - loss: 0.0655 - accuracy: 0.9856
Epoch 425/500
900/900 [==============================] - 0s 81us/sample - loss: 0.0498 - accuracy: 0.9900
Epoch 426/500
900/900 [==============================] - 0s 84us/sample - loss: 0.0933 - accuracy: 0.9700
Epoch 427/500
900/900 [==============================] - 0s 82us/sample - loss: 0.2059 - accuracy: 0.9289
Epoch 428/500
900/900 [==============================] - 0s 

900/900 [==============================] - 0s 81us/sample - loss: 0.9228 - accuracy: 0.6856
Epoch 45/500
900/900 [==============================] - 0s 84us/sample - loss: 0.8960 - accuracy: 0.7056
Epoch 46/500
900/900 [==============================] - 0s 81us/sample - loss: 0.8772 - accuracy: 0.7056
Epoch 47/500
900/900 [==============================] - 0s 89us/sample - loss: 0.8817 - accuracy: 0.7056
Epoch 48/500
900/900 [==============================] - 0s 93us/sample - loss: 0.8873 - accuracy: 0.6978
Epoch 49/500
900/900 [==============================] - 0s 98us/sample - loss: 0.8797 - accuracy: 0.7100
Epoch 50/500
900/900 [==============================] - 0s 91us/sample - loss: 0.8893 - accuracy: 0.6989
Epoch 51/500
900/900 [==============================] - 0s 106us/sample - loss: 0.8631 - accuracy: 0.7089
Epoch 52/500
900/900 [==============================] - 0s 104us/sample - loss: 0.8538 - accuracy: 0.7200
Epoch 53/500
900/900 [==============================] - 0s 91us/sa

900/900 [==============================] - 0s 83us/sample - loss: 0.3456 - accuracy: 0.8922
Epoch 199/500
900/900 [==============================] - 0s 93us/sample - loss: 0.3438 - accuracy: 0.8944
Epoch 200/500
900/900 [==============================] - 0s 82us/sample - loss: 0.3743 - accuracy: 0.8789
Epoch 201/500
900/900 [==============================] - 0s 76us/sample - loss: 0.3853 - accuracy: 0.8700
Epoch 202/500
900/900 [==============================] - 0s 81us/sample - loss: 0.3507 - accuracy: 0.8889
Epoch 203/500
900/900 [==============================] - 0s 80us/sample - loss: 0.3477 - accuracy: 0.8856
Epoch 204/500
900/900 [==============================] - 0s 78us/sample - loss: 0.3617 - accuracy: 0.8822
Epoch 205/500
900/900 [==============================] - 0s 79us/sample - loss: 0.3483 - accuracy: 0.8867
Epoch 206/500
900/900 [==============================] - 0s 81us/sample - loss: 0.3379 - accuracy: 0.8944
Epoch 207/500
900/900 [==============================] - 0s 

900/900 [==============================] - 0s 90us/sample - loss: 0.1180 - accuracy: 0.9744
Epoch 353/500
900/900 [==============================] - 0s 82us/sample - loss: 0.1377 - accuracy: 0.9644
Epoch 354/500
900/900 [==============================] - 0s 78us/sample - loss: 0.1552 - accuracy: 0.9544
Epoch 355/500
900/900 [==============================] - 0s 81us/sample - loss: 0.1115 - accuracy: 0.9744
Epoch 356/500
900/900 [==============================] - 0s 78us/sample - loss: 0.1574 - accuracy: 0.9533
Epoch 357/500
900/900 [==============================] - 0s 82us/sample - loss: 0.1012 - accuracy: 0.9778
Epoch 358/500
900/900 [==============================] - 0s 79us/sample - loss: 0.0872 - accuracy: 0.9900
Epoch 359/500
900/900 [==============================] - 0s 76us/sample - loss: 0.1111 - accuracy: 0.9756
Epoch 360/500
900/900 [==============================] - 0s 83us/sample - loss: 0.0957 - accuracy: 0.9822
Epoch 361/500
900/900 [==============================] - 0s 

In [10]:
print("\n %.f fold accuracy:" % n_fold, accuracy)


 10 fold accuracy: ['0.7300', '0.6700', '0.7100', '0.7000', '0.6600', '0.6300', '0.6600', '0.5900', '0.7200', '0.6500']


In [11]:
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================


 Accuracy: 0.9640
